In [ ]:
!pip install transformers datasets torch
!pip install accelerate -U

In [1]:
import os
import re
from transformers import GPT2Tokenizer

def preprocess_data(input_file, output_file, tokenizer_name="gpt2"):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_name)
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    with open(output_file, 'w', encoding='utf-8') as f:
        for line in lines:
            line = line.lower()

            # Tokenize the line
            token_ids = tokenizer.encode(line, add_special_tokens=False)
            # Convert token IDs back to tokens
            tokenized_line = tokenizer.convert_ids_to_tokens(token_ids)
            # Convert tokens to text and remove special tokens
            processed_line = " ".join(tokenized_line).replace('Ġ', '').replace('Ċ', '').replace('�', '').strip()

            processed_line = re.sub(r'[^a-zA-Z0-9\s,():%&]', '', processed_line)
            processed_line = re.sub(r'\bgu vi\b', 'guvi', processed_line)

            # Remove extra spaces
            processed_line = re.sub(r'\s+', ' ', processed_line)
            # Write the processed line to the output file
            f.write(processed_line + "\n")

input_file = r"C:\Users\ss996\OneDrive\Desktop\Projects\P6-guvi\guvi_data.txt"  # Make sure this path is correct
output_file = "processed_data.txt"
preprocess_data(input_file, output_file)

d:\Anaconda\envs\python3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# Load pre-trained model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Create dataset
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )

train_dataset = load_dataset(output_file, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

# 6689a1b60c0e560ff167d6fbb07df3f499b21d6b

Error while downloading from https://cdn-lfs.hf.co/gpt2/248dfc3911869ec493c76e65bf2fcf7f615828b0254c12b473182f0f81d3a707?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1736049016&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNjA0OTAxNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9ncHQyLzI0OGRmYzM5MTE4NjllYzQ5M2M3NmU2NWJmMmZjZjdmNjE1ODI4YjAyNTRjMTJiNDczMTgyZjBmODFkM2E3MDc%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=FZcFl3s5swRKs86lATqJmax9BhjWC6cIoJemP0lB0FeZtNnTVA7Uz9jVtKC%7EFy-I046xzS8XqlAxXcNV-2hsEy6I-u%7Ey8lmK%7E75GUdTbTyQOsmCWQkUAt9C%7EihRg4SF0Iz1jtomzidCBDxG9FUz80FupH4-DQN9FWTecc8pvXkE2eQtsT6ZAs27EBNQOvODPknzPzRvzNMIlYhl%7Ex0iO9C0-zwvJujanr8eRAXExPnVb6NGlTazvjrMcVK2tR3ww7ao0NfmjqjJ8iugHlV1Fa8gtiS5%7EGHK-8uz6S0RjQUeLYmI6LXYazBSu3B6I2r2sP2w88in9yZDyA8pFUanAxw__&Key-Pair-Id=K3RPWS32NSSJCE: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decrypt

{'train_runtime': 1094.811, 'train_samples_per_second': 1.247, 'train_steps_per_second': 0.315, 'train_loss': 3.306143498075181, 'epoch': 5.0}


('./fine_tuned_model\\tokenizer_config.json',
 './fine_tuned_model\\special_tokens_map.json',
 './fine_tuned_model\\vocab.json',
 './fine_tuned_model\\merges.txt',
 './fine_tuned_model\\added_tokens.json')

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the fine-tuned model and tokenizer
model_name_or_path = "./fine_tuned_model"
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)

# Set the pad_token to eos_token if it's not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Define the text generation function
def generate_text(model, tokenizer, seed_text, max_length=100, temperature=1.0, num_return_sequences=1):
    # Tokenize the input text with padding
    inputs = tokenizer(seed_text, return_tensors='pt', padding=True, truncation=True)

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Generate text
    with torch.no_grad():
        output = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            temperature=temperature,
            num_return_sequences=num_return_sequences,
            do_sample=True,
            top_k=50,
            top_p=0.50,
            pad_token_id=tokenizer.eos_token_id  # Ensure padding token is set to eos_token_id
        )

    # Decode the generated text
    generated_texts = []
    for i in range(num_return_sequences):
        generated_text = tokenizer.decode(output[i], skip_special_tokens=True)
        generated_texts.append(generated_text)

    return generated_texts

# Test the model
seed_text = input("Enter seed text: ")
generated_texts = generate_text(model, tokenizer, seed_text, max_length=50, temperature=0.001, num_return_sequences=1)

for i, text in enumerate(generated_texts):
    print(f"Generated Text {i + 1}:\n{text}\n")

Generated Text 1:
guvi is a free online learning platform that offers courses in various languages , including english , tam il , guvi offers courses in various languages , including english , tam il , guvi offers courses in various languages , including tam il , guvi offers

